In [8]:
# -*- coding: utf-8 -*-
from get_response import PoliGPT
from typing import Any, Dict, List, Sequence, Mapping # Mapping es más general que Dict
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    # Considera añadir context_recall si tienes la columna 'reference'/'ground_truth'
    # context_recall,
)
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings

# --- NECESITAS DEFINIR ESTAS RESPUESTAS DE REFERENCIA ---
# Estas son las respuestas "correctas" o "ideales" para cada pregunta.
# Tendrás que escribirlas tú mismo basándote en el conocimiento esperado.
# Ejemplo (DEBES RELLENARLAS CON LAS RESPUESTAS CORRECTAS REALES):
ground_truths: Dict[str, str] = {
    "¿Es necesario el conocimiento de Valenciano para acceder a una plaza en la Universidad?": "La normativa actual sobre el requisito lingüístico para el acceso a la función pública docente y su aplicación específica en las universidades de la Comunitat Valenciana indica que [respuesta detallada aquí...]",
    "¿Cuáles son los requisitos para estudiar en la Universidad?": "Para acceder a estudios universitarios oficiales de Grado se requiere cumplir con [requisitos como EBAU, CFGS, mayores de 25/40/45, titulación previa...]",
    "¿Qué carreras se ofrecen en la Universidad?": "La universidad [Nombre de la Universidad, si es específica] ofrece una amplia variedad de grados en áreas como [mencionar áreas principales: Ciencias, Humanidades, Ingeniería, Salud, Sociales...] incluyendo [mencionar algunos ejemplos específicos si es posible].",
    "¿Hay becas disponibles para estudiantes?": "Sí, existen diversas becas y ayudas al estudio tanto del Ministerio de Educación como de la Generalitat Valenciana y de la propia universidad, como [mencionar tipos: matrícula, excelencia, movilidad, socioeconómicas...].",
    "¿Qué tipo de apoyo académico se ofrece a los estudiantes?": "Se ofrecen servicios de apoyo como tutorías personalizadas, programas de mentoría, cursos de refuerzo, talleres de técnicas de estudio, y acceso a bibliotecas y recursos digitales.",
    "¿Cómo se puede acceder a la información sobre las becas?": "La información detallada sobre convocatorias, requisitos y plazos de solicitud de becas se puede encontrar en la página web oficial de la universidad, en la sección de 'Becas y Ayudas', así como en los portales del Ministerio de Educación y la Conselleria de Educación de la Generalitat Valenciana.",
}
# ---------------------------------------------------------


def build_eval_dataset(
    client: "PoliGPT",
    questions: Sequence[str],
    ground_truths: Mapping[str, str], # Añadido: Acepta las respuestas de referencia
    k_context: int = 3
) -> Dataset:
    """Construye un *Dataset* de `datasets` compatible con *Ragas*.

    Args:
        client: Instancia de :class:`PoliGPT`.
        questions: Lista de preguntas a evaluar.
        ground_truths: Diccionario mapeando preguntas a sus respuestas de referencia.
        k_context: Nº de fragmentos de contexto que se solicitarán a PoliGPT.

    Returns:
        Dataset con columnas 'question', 'contexts', 'answer' y 'reference'.
    """
    rows: Dict[str, List[Any]] = {
        "question": [],
        "contexts": [],
        "answer": [],
        "reference": [] # Añadida la columna para las respuestas de referencia
    }

    for q in questions:
        # Asegúrate de que hay una respuesta de referencia para esta pregunta
        if q not in ground_truths:
            print(f"Advertencia: No se encontró respuesta de referencia (ground truth) para la pregunta: '{q}'. Saltando...")
            continue

        res = client.query_poligpt(q, k_context=k_context)

        # Salta en caso de error en la consulta a PoliGPT
        if "error" in res:
            print(f"Pregunta ignorada por falta de contexto o error en PoliGPT: '{q}'")
            continue

        rows["question"].append(q)
        rows["answer"].append(res["response"])
        rows["contexts"].append([c["content"] for c in res["contexts"]])
        rows["reference"].append(ground_truths[q]) # Añade la respuesta de referencia correspondiente

    # Verifica que todas las listas tengan la misma longitud antes de crear el Dataset
    list_lengths = {key: len(value) for key, value in rows.items()}
    if len(set(list_lengths.values())) > 1:
        print("Error: Las listas de columnas tienen diferentes longitudes:", list_lengths)
        # Decide cómo manejar esto: ¿lanzar un error, devolver un dataset vacío, etc.?
        # Por ahora, simplemente imprimimos y devolvemos un dataset vacío o parcial
        # Podrías intentar filtrar las filas incompletas si es apropiado
        non_empty_rows = {k: v for k, v in rows.items() if v}
        if not non_empty_rows or len(set(len(lst) for lst in non_empty_rows.values())) > 1 :
             return Dataset.from_dict({"question":[], "contexts":[], "answer":[], "reference":[]}) # O lanzar excepción
        # Intentar crear con las columnas que sí tienen datos (si todas tienen la misma longitud > 0)
        min_len = min(len(v) for v in non_empty_rows.values())
        rows_truncated = {k: v[:min_len] for k, v in non_empty_rows.items()}
        print("Advertencia: Creando dataset con datos truncados debido a longitudes desiguales.")
        return Dataset.from_dict(rows_truncated)


    # Si no hay datos válidos después de filtrar, devuelve un dataset vacío
    if not rows["question"]:
         print("Advertencia: No se generaron filas válidas para el dataset de evaluación.")
         return Dataset.from_dict({"question":[], "contexts":[], "answer":[], "reference":[]})


    return Dataset.from_dict(rows)


def ragas_evaluate_local(eval_ds: Dataset): # Renombrada para claridad
    """Evalúa un *Dataset* usando modelos locales vía Ollama."""
    if eval_ds.num_rows == 0:
        print("El dataset de evaluación está vacío. No se puede evaluar.")
        return {}

    # --- Configurar LLM y Embeddings de Ollama ---
    # Asegúrate de que el servidor Ollama esté corriendo en segundo plano
    # Reemplaza 'llama3' con el modelo que descargaste y quieres usar para juzgar
    ragas_llm = ChatOllama(model="llama3")

    # Puedes usar el mismo modelo para embeddings o uno específico como 'nomic-embed-text'
    # Si no especificas 'model', OllamaEmbeddings puede usar un modelo por defecto
    ragas_embeddings = OllamaEmbeddings(model="nomic-embed-text") # o "llama3", etc.
    # -------------------------------------------

    metrics = [
        # Ten en cuenta que la calidad de la evaluación dependerá del modelo local
        faithfulness,
        answer_relevancy,
        context_precision,
        # context_recall, # Descomentado si tienes la columna 'reference'
    ]
    column_map = {"reference": "ground_truth"}

    print(f"Evaluando con el LLM local: {ragas_llm.model}")
    print(f"Evaluando con los Embeddings locales: {ragas_embeddings.model}")
    print("Usando mapeo de columnas:", column_map)

    # --- Pasar LLM y Embeddings a Ragas ---
    results = evaluate(
        eval_ds,
        metrics=metrics,
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        column_map=column_map,
        raise_exceptions=False # Importante: Modelos locales pueden fallar más a menudo
                              # False intentará continuar, True parará en el primer error
    )
    # ---------------------------------------
    return results

In [9]:
# --- Script Principal ---

preguntas = [
    "¿Es necesario el conocimiento de Valenciano para acceder a una plaza en la Universidad?",
    "¿Cuáles son los requisitos para estudiar en la Universidad?",
    "¿Qué carreras se ofrecen en la Universidad?",
    "¿Hay becas disponibles para estudiantes?",
    "¿Qué tipo de apoyo académico se ofrece a los estudiantes?",
    "¿Cómo se puede acceder a la información sobre las becas?",
]

# Cargar el cliente PoliGPT
# Asegúrate de que la ruta al índice FAISS sea correcta
try:
    poligpt = PoliGPT(faiss_index_dir='../01_data/project_faiss')
except Exception as e:
    print(f"Error al cargar PoliGPT: {e}")
    # Salir o manejar el error adecuadamente
    exit()

# Construir el dataset de evaluación, pasando las respuestas de referencia
eval_ds = build_eval_dataset(poligpt, preguntas, ground_truths, k_context=3)

# Verificar si el dataset se construyó correctamente
print("Dataset construido:")
print(eval_ds)

for i in range(len(eval_ds)):
    print(f"Pregunta: {eval_ds['question'][i]}")
    print(f"Respuesta: {eval_ds['answer'][i]}")
    print(f"Contextos: {eval_ds['contexts'][i]}")
    print(f"Referencia: {eval_ds['reference'][i]}")
    print("-" * 40)


# Evaluar el dataset con la función LOCAL
if eval_ds.num_rows > 0:
    print("\nIniciando evaluación con modelos locales (Ollama)...")
    try:
        # Llamar a la función que usa Ollama
        results = ragas_evaluate_local(eval_ds)
        print("\nResultados de la evaluación:")
        if results:
             for metric, score in results.items():
                 # Manejar posibles None si raise_exceptions=False
                 score_display = f"{score:.4f}" if score is not None else "N/A (error)"
                 print(f"{metric}: {score_display}")
        else:
            print("La evaluación no devolvió resultados.")

    except Exception as e:
        print(f"\nError durante la evaluación local con Ragas: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\nNo se pudo evaluar porque el dataset está vacío o no se pudo construir.")


FAISS inicializado - Vectores: 79044 | Dimensión: 768
Dataset construido:
Dataset({
    features: ['question', 'contexts', 'answer', 'reference'],
    num_rows: 6
})
Pregunta: ¿Es necesario el conocimiento de Valenciano para acceder a una plaza en la Universidad?
Respuesta: No se menciona la necesidad de conocimiento de Valenciano para acceder a una plaza en la Universidad en el contexto proporcionado.
Contextos: ['València han d’estar adscrits: \ni. Al departament  de la plaça oferida en el concurs, \nen qualsevol  àrea de coneixement  d’aquest.  \n \nii. O a la mateixa àrea de coneixement  o \nàrea afí a la \nplaça oferida, si estan adscrits a un altre \ndepartament  de la Universitat  Politècnica  de \nValència. \nII. Els professors  que no pertanyen  a la Universitat  \nPolitècnica  de València han d’estar adscrits a l’àrea o \nàrees afins del perfil de la plaça.', 'aquesta formació plurilingüe  presenta \nl’avantatge  d’augmentar  les oportunitats  \nd’ocupabilitat  dels diplomats

Traceback (most recent call last):
  File "/var/folders/1w/2mpv0jw11qj_v4tpnrdqv7t40000gn/T/ipykernel_7491/2142984142.py", line 38, in <module>
    results = ragas_evaluate(eval_ds)
              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/1w/2mpv0jw11qj_v4tpnrdqv7t40000gn/T/ipykernel_7491/3839284683.py", line 117, in ragas_evaluate
    return evaluate(eval_ds, metrics=metrics)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mhurben/anaconda3/envs/proy/lib/python3.11/site-packages/ragas/_analytics.py", line 227, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mhurben/anaconda3/envs/proy/lib/python3.11/site-packages/ragas/evaluation.py", line 198, in evaluate
    llm = llm_factory()
          ^^^^^^^^^^^^^
  File "/Users/mhurben/anaconda3/envs/proy/lib/python3.11/site-packages/ragas/llms/base.py", line 416, in llm_factory
    openai_model = ChatOpenAI(
                   ^^^^^^^^^^^
  File "/Users/mhurben/anaconda3/envs/proy/